In [141]:
import pandas as pd

acc = []
for i in range(8):
    acc.append(pd.read_csv(f'./com_scores_lies_split_{i}.csv', dtype={'model_bad_id': 'object', 'model_good_id': 'object'}))

df_raw = pd.concat(acc)

In [142]:
df = df_raw.drop(['caller/truth_ratio', 'caller/Messages.CARICO_NOT_BRISCOLA_True',
       'caller/Messages.CARICO_NOT_BRISCOLA_False',
       'caller/Messages.BRISCOLINO_True', 'caller/Messages.BRISCOLINO_False',
       'caller/Messages.BRISCOLA_FIGURA_True',
       'caller/Messages.BRISCOLA_FIGURA_False',
       'caller/Messages.BRISCOLA_CARICO_True',
       'caller/Messages.BRISCOLA_CARICO_False', 'caller/Messages.LISCIO_True',
       'caller/Messages.LISCIO_False', 'callee/truth_ratio',
       'callee/Messages.CARICO_NOT_BRISCOLA_True',
       'callee/Messages.CARICO_NOT_BRISCOLA_False',
       'callee/Messages.BRISCOLINO_True', 'callee/Messages.BRISCOLINO_False',
       'callee/Messages.BRISCOLA_FIGURA_True',
       'callee/Messages.BRISCOLA_FIGURA_False',
       'callee/Messages.BRISCOLA_CARICO_True',
       'callee/Messages.BRISCOLA_CARICO_False', 'callee/Messages.LISCIO_True',
       'callee/Messages.LISCIO_False', 'good/truth_ratio',
       'good/Messages.CARICO_NOT_BRISCOLA_True',
       'good/Messages.CARICO_NOT_BRISCOLA_False',
       'good/Messages.BRISCOLINO_True', 'good/Messages.BRISCOLINO_False',
       'good/Messages.BRISCOLA_FIGURA_True',
       'good/Messages.BRISCOLA_FIGURA_False',
       'good/Messages.BRISCOLA_CARICO_True',
       'good/Messages.BRISCOLA_CARICO_False', 'good/Messages.LISCIO_True',
       'good/Messages.LISCIO_False'], axis=1)

In [143]:
df2 = pd.merge(df, df, left_on=['model_bad_id', 'model_good_id'], right_on=['model_good_id', 'model_bad_id'], suffixes=('_A', '_B'))
df2.drop(['model_bad_name_A', 'model_bad_step_A', 'model_good_name_A', 'model_good_step_A',  'model_bad_name_B', 'model_bad_step_B', 'model_good_name_B',
       'model_good_step_B'], axis=1, inplace=True)

df2.rename(columns={"model_bad_id_A": "model_id", "model_good_id_A": "adversary_id", "reward_bad_A": "bad_reward", "reward_good_B": "good_reward"}, inplace=True)
df2.drop(['reward_good_A', 'model_bad_id_B', 'model_good_id_B', 'reward_bad_B'], axis=1, inplace=True)
df2 = df2[df2['model_id'] != df2['adversary_id']]

In [144]:
df2

,model_id,adversary_id,bad_reward,good_reward
1,0,1,89.142,37.078
2,0,2,89.444,35.302
3,0,3,89.565,35.332
4,0,4,88.580,34.536
5,0,5,88.623,41.800
...,...,...,...,...
15369,121,heuristic,84.959,26.885
15370,random,121,72.050,29.898
15371,heuristic,121,93.115,35.041
15372,random,heuristic,72.542,30.531


In [145]:
def compute_offset(elos, a, b, scoreBad, scoreGood):
    eloA = elos[elos['model_id'] == a]['elo'].values[0]
    eloB = elos[elos['model_id'] == b]['elo'].values[0]
    S = (scoreBad + scoreGood) / 2
    E = 120 / (1 + 10 ** ((eloB - eloA) / 3500))
    return 25 * (S - E)

In [146]:
elos = pd.DataFrame({'model_id': df['model_bad_id'].unique(), 'elo': 1000})
for _ in range(20):
    elos_tmp = elos.copy()
    for i in range(len(elos)):
        name = elos['model_id'].iloc[i]
        offsets = df2[df2['model_id'] == name].apply(lambda x: compute_offset(elos, x['model_id'], x['adversary_id'], x['bad_reward'], x['good_reward']), axis=1)
        eloA = elos.iloc[i]['elo']
        elos_tmp.loc[i, 'elo'] = eloA + (offsets.sum() / len(offsets))
    elos = elos_tmp

In [147]:
elos_named = pd.merge(elos, df.drop(['model_good_id', 'model_good_name', 'model_good_step', 'reward_bad', 'reward_good'], axis=1).drop_duplicates(), left_on='model_id', right_on='model_bad_id', suffixes=('_elo', '_df'), how='inner')
elos_named.rename(columns={"model_bad_id": "name", "model_bad_step": "step"}, inplace=True)
elos_named['model_bad_name'] = elos_named['model_bad_name'].apply(lambda x: x.replace('2nd phase ', ''))
elos_named = elos_named.sort_values(by=['elo'], ascending=False)
elos_named

,model_id,elo,name,model_bad_name,step
0,0,1358.348911,0,COMM RNN h=32 m=64 P2,192000000
123,heuristic,1354.971987,heuristic,heuristic,0
78,78,1333.272533,78,COMM RNN h=32 m=64 P2,160000000
77,77,1326.954736,77,COMM RNN h=32 m=64 P2,153600000
76,76,1323.496722,76,COMM RNN h=32 m=64 P2,128000000
...,...,...,...,...,...
80,80,629.926649,80,COMM RNN h=128 m=64 P1,51200000
100,100,585.187834,100,COMM RNN h=32 m=128 P1,25600000
86,86,582.003025,86,COMM RNN h=64 m=128 P1,25600000
93,93,504.990747,93,COMM RNN h=64 m=64 P1,25600000


In [148]:
table = elos_named.groupby('model_bad_name').max('elo')
table.sort_values(by=['elo'], ascending=False)

table_P2 = table.copy()
table_P2.reset_index(inplace=True)
table_P2['model_bad_name'] = table_P2['model_bad_name'].apply(lambda x: x.replace('P2', ''))
table_P2['elo'] = table_P2['elo'].round(0).astype(int)
table_P2 = table_P2.sort_values(by=['elo'], ascending=False)
table_P2

,model_bad_name,elo,step
11,COMM RNN h=32 m=64,1358,192000000
16,heuristic,1355,0
13,COMM RNN h=64 m=128,1317,192000000
1,COMM NN h=32 sample_batch=True,1293,192000000
15,COMM RNN h=64 m=64,1265,192000000
7,COMM RNN h=128 m=64,1260,192000000
9,COMM RNN h=32 m=128,1254,192000000
0,COMM NN h=32 sample_batch=False,1246,192000000
5,COMM RNN h=128 m=128,1238,160000256
2,COMM NN h=64 sample_batch=False,1172,192000000


In [149]:
print(table_P2.to_latex(index=False))

\begin{tabular}{lrr}
\toprule
model_bad_name & elo & step \\
\midrule
COMM RNN h=32 m=64  & 1358 & 192000000 \\
heuristic & 1355 & 0 \\
COMM RNN h=64 m=128  & 1317 & 192000000 \\
COMM NN h=32 sample_batch=True  & 1293 & 192000000 \\
COMM RNN h=64 m=64  & 1265 & 192000000 \\
COMM RNN h=128 m=64  & 1260 & 192000000 \\
COMM RNN h=32 m=128  & 1254 & 192000000 \\
COMM NN h=32 sample_batch=False  & 1246 & 192000000 \\
COMM RNN h=128 m=128  & 1238 & 160000256 \\
COMM NN h=64 sample_batch=False  & 1172 & 192000000 \\
COMM NN h=64 sample_batch=True  & 1143 & 192000000 \\
COMM RNN h=64 m=128 P1 & 849 & 160000000 \\
COMM RNN h=128 m=64 P1 & 831 & 160000000 \\
COMM RNN h=64 m=64 P1 & 822 & 160000000 \\
COMM RNN h=32 m=128 P1 & 811 & 160000000 \\
COMM RNN h=128 m=128 P1 & 772 & 160000000 \\
COMM RNN h=32 m=64 P1 & 765 & 160000000 \\
random & 719 & 0 \\
\bottomrule
\end{tabular}



In [150]:
import plotly.express as px
import plotly.graph_objects as go


In [151]:
def save_fig(fig, name):
    fig.update_layout(legend_title_text='', showlegend=True)
    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))

    fig.update_layout(width =500, height=220, 
                    font_family="Serif", font_size=11, title_font_size=13, 
                    margin_l=5, margin_t=1, margin_b=1, margin_r=5)

    fig.update_layout(title="")
    fig.update_yaxes(nticks=12)
    import plotly.io as pio
    #save a figure of 300dpi, width 1.5 inches, height 0.75inches
    pio.write_image(fig, f"./img/{name}.png")
    fig.show()

In [154]:
elos_named

,model_id,elo,name,model_bad_name,step
0,0,1358.348911,0,COMM RNN h=32 m=64 P2,192000000
123,heuristic,1354.971987,heuristic,heuristic,0
78,78,1333.272533,78,COMM RNN h=32 m=64 P2,160000000
77,77,1326.954736,77,COMM RNN h=32 m=64 P2,153600000
76,76,1323.496722,76,COMM RNN h=32 m=64 P2,128000000
...,...,...,...,...,...
80,80,629.926649,80,COMM RNN h=128 m=64 P1,51200000
100,100,585.187834,100,COMM RNN h=32 m=128 P1,25600000
86,86,582.003025,86,COMM RNN h=64 m=128 P1,25600000
93,93,504.990747,93,COMM RNN h=64 m=64 P1,25600000


In [152]:
plots_elos = elos_named.copy()
plots_elos['model_bad_name'] = plots_elos['model_bad_name'].apply(lambda x: x.replace('2nd phase ', ''))

In [155]:
fig_nn_p2 = go.Figure()
fig_rnn_p2 = go.Figure()
fig_rnn_p1 = go.Figure()
for name, group in plots_elos.groupby('model_bad_name'):    
    group.loc[-1] = [1000000, 718.526895, 'random', 'random', 0]
    group = group.sort_values(by=['step'])

    if name == 'COMM RNN h=128 m=128 P2':
        group = group.drop([11,12])
        group.iloc[-1]['step'] = 151040000

    name = name.replace('COMM ', '')
    
    if 'RNN' in name and 'P1' in name:
        fig = fig_rnn_p1
    elif 'RNN' in name and 'P2' in name:
        fig = fig_rnn_p2
    elif 'NN' in name and 'P2' in name:
        fig = fig_nn_p2
   
    if name == 'random' or name == 'heuristic':
        continue
   

    display_name = name
    if 'RNN' not in display_name:
        display_name = display_name.replace('NN', 'MLP')
        display_name = display_name.replace('sample_batch', 'sb')
  
    fig.add_trace(go.Scatter(x=group['step']/16, y=group['elo'], name=display_name))
   

fig_nn_p2.add_hline(y=718.526895, line_dash="dot", line_color="gray", annotation_text='random')
fig_nn_p2.add_hline(y=1354.971987, line_dash="dot", line_color="green", annotation_text='heuristic', annotation_position="top left")
fig_nn_p2.add_vline(x=10000000, line_dash="dash", line_color="gray", annotation_text='2nd phase', annotation_position="bottom right", annotation_textangle = 90,)
fig_nn_p2.update_layout(title=f'ELO score over time MLP P2', xaxis_title='Number of games', yaxis_title='ELO score')
fig_nn_p2.show()
save_fig(fig_nn_p2, 'com_elo_mlp_p2')

fig_rnn_p2.add_hline(y=718.526895, line_dash="dot", line_color="gray", annotation_text='random')
fig_rnn_p2.add_hline(y=1354.971987, line_dash="dot", line_color="green", annotation_text='heuristic', annotation_position="top left")

fig_rnn_p2.add_vline(x=8000000, line_dash="dash", line_color="gray", annotation_text='*', annotation_position="bottom right", annotation_textangle = 90)
fig_rnn_p2.add_vline(x=10000000, line_dash="dash", line_color="gray", annotation_text='2nd phase', annotation_position="bottom right", annotation_textangle = 90)
fig_rnn_p2.update_layout(title=f'ELO score over time RNN P2', xaxis_title='Number of games', yaxis_title='ELO score')
fig_rnn_p2.show()
save_fig(fig_rnn_p2, 'com_elo_rnn_p2')

fig_rnn_p1.add_hline(y=718.526895, line_dash="dot", line_color="gray", annotation_text='random')
fig_rnn_p1.add_hline(y=1354.971987, line_dash="dot", line_color="green", annotation_text='heuristic', annotation_position="top left")
fig_rnn_p1.update_layout(title=f'ELO score over time MLP P1', xaxis_title='Number of games', yaxis_title='ELO score')
fig_rnn_p1.show()
save_fig(fig_rnn_p1, 'com_elo_rnn_p1')

/tmp/ipykernel_7305/2134489893.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df_raw.columns

Index(['model_bad_id', 'model_good_id', 'model_bad_name', 'model_bad_step',
       'model_good_name', 'model_good_step', 'reward_bad', 'reward_good',
       'caller/truth_ratio', 'caller/Messages.CARICO_NOT_BRISCOLA_True',
       'caller/Messages.CARICO_NOT_BRISCOLA_False',
       'caller/Messages.BRISCOLINO_True', 'caller/Messages.BRISCOLINO_False',
       'caller/Messages.BRISCOLA_FIGURA_True',
       'caller/Messages.BRISCOLA_FIGURA_False',
       'caller/Messages.BRISCOLA_CARICO_True',
       'caller/Messages.BRISCOLA_CARICO_False', 'caller/Messages.LISCIO_True',
       'caller/Messages.LISCIO_False', 'callee/truth_ratio',
       'callee/Messages.CARICO_NOT_BRISCOLA_True',
       'callee/Messages.CARICO_NOT_BRISCOLA_False',
       'callee/Messages.BRISCOLINO_True', 'callee/Messages.BRISCOLINO_False',
       'callee/Messages.BRISCOLA_FIGURA_True',
       'callee/Messages.BRISCOLA_FIGURA_False',
       'callee/Messages.BRISCOLA_CARICO_True',
       'callee/Messages.BRISCOLA_CARICO_Fa

In [ ]:
bad_roles = df_raw.groupby('model_bad_id').mean(['caller/truth_ratio', 'caller/Messages.CARICO_NOT_BRISCOLA_True',
       'caller/Messages.CARICO_NOT_BRISCOLA_False',
       'caller/Messages.BRISCOLINO_True', 'caller/Messages.BRISCOLINO_False',
       'caller/Messages.BRISCOLA_FIGURA_True',
       'caller/Messages.BRISCOLA_FIGURA_False',
       'caller/Messages.BRISCOLA_CARICO_True',
       'caller/Messages.BRISCOLA_CARICO_False', 'caller/Messages.LISCIO_True',
       'caller/Messages.LISCIO_False', 'callee/truth_ratio',
       'callee/Messages.CARICO_NOT_BRISCOLA_True',
       'callee/Messages.CARICO_NOT_BRISCOLA_False',
       'callee/Messages.BRISCOLINO_True', 'callee/Messages.BRISCOLINO_False',
       'callee/Messages.BRISCOLA_FIGURA_True',
       'callee/Messages.BRISCOLA_FIGURA_False',
       'callee/Messages.BRISCOLA_CARICO_True',
       'callee/Messages.BRISCOLA_CARICO_False', 'callee/Messages.LISCIO_True',
       'callee/Messages.LISCIO_False'])
good_roles = df_raw.groupby('model_good_id').mean(['good/truth_ratio',
       'good/Messages.CARICO_NOT_BRISCOLA_True',
       'good/Messages.CARICO_NOT_BRISCOLA_False',
       'good/Messages.BRISCOLINO_True', 'good/Messages.BRISCOLINO_False',
       'good/Messages.BRISCOLA_FIGURA_True',
       'good/Messages.BRISCOLA_FIGURA_False',
       'good/Messages.BRISCOLA_CARICO_True',
       'good/Messages.BRISCOLA_CARICO_False', 'good/Messages.LISCIO_True',
       'good/Messages.LISCIO_False'])
bad_roles.reset_index(inplace=True)
good_roles.reset_index(inplace=True)
bad_roles.drop(['good/truth_ratio',
       'good/Messages.CARICO_NOT_BRISCOLA_True',
       'good/Messages.CARICO_NOT_BRISCOLA_False',
       'good/Messages.BRISCOLINO_True', 'good/Messages.BRISCOLINO_False',
       'good/Messages.BRISCOLA_FIGURA_True',
       'good/Messages.BRISCOLA_FIGURA_False',
       'good/Messages.BRISCOLA_CARICO_True',
       'good/Messages.BRISCOLA_CARICO_False', 'good/Messages.LISCIO_True',
       'good/Messages.LISCIO_False'], axis=1, inplace=True)
good_roles.drop(['caller/truth_ratio',
       'caller/Messages.CARICO_NOT_BRISCOLA_True',
       'caller/Messages.CARICO_NOT_BRISCOLA_False',
       'caller/Messages.BRISCOLINO_True', 'caller/Messages.BRISCOLINO_False',
       'caller/Messages.BRISCOLA_FIGURA_True',
       'caller/Messages.BRISCOLA_FIGURA_False',
       'caller/Messages.BRISCOLA_CARICO_True',
       'caller/Messages.BRISCOLA_CARICO_False', 'caller/Messages.LISCIO_True',
       'caller/Messages.LISCIO_False', 'callee/truth_ratio',
       'callee/Messages.CARICO_NOT_BRISCOLA_True',
       'callee/Messages.CARICO_NOT_BRISCOLA_False',
       'callee/Messages.BRISCOLINO_True', 'callee/Messages.BRISCOLINO_False',
       'callee/Messages.BRISCOLA_FIGURA_True',
       'callee/Messages.BRISCOLA_FIGURA_False',
       'callee/Messages.BRISCOLA_CARICO_True',
       'callee/Messages.BRISCOLA_CARICO_False', 'callee/Messages.LISCIO_True',
       'callee/Messages.LISCIO_False'], axis=1, inplace=True)

In [ ]:
idx = elos_named.groupby('model_bad_name')['elo'].idxmax()
elos_max = elos_named.loc[idx,]
elos_max

,model_id,elo,name,model_bad_name,step
26,26,1245.837715,26,COMM NN h=32 sample_batch=False P2,160000000
6,6,1292.638855,6,COMM NN h=32 sample_batch=True P2,192000000
8,8,1171.546162,8,COMM NN h=64 sample_batch=False P2,192000000
5,5,1143.440756,5,COMM NN h=64 sample_batch=True P2,192000000
120,120,771.830792,120,COMM RNN h=128 m=128 P1,160000000
121,121,1238.441849,121,COMM RNN h=128 m=128 P2,143360000
84,84,830.710862,84,COMM RNN h=128 m=64 P1,153600000
3,3,1259.962503,3,COMM RNN h=128 m=64 P2,192000000
105,105,810.610743,105,COMM RNN h=32 m=128 P1,153600000
1,1,1254.069632,1,COMM RNN h=32 m=128 P2,192000000


In [ ]:
m = pd.merge(elos_max, bad_roles, left_on='model_id', right_on='model_bad_id')
m = pd.merge(m, good_roles, left_on='model_id', right_on='model_good_id')
m.drop(['model_bad_step_x', 'model_good_step_x', 'model_good_id', 
        'model_bad_step_y', 'reward_bad_x', 'reward_good_x', 
        'reward_bad_y', 'reward_good_y', 'model_good_step_x', 'model_good_step_y',], axis=1, inplace=True)

table_paper = m[['model_bad_name','elo','caller/truth_ratio', 'good/truth_ratio', 'callee/truth_ratio']]
table_paper['caller/bluff_ratio'] = 1 - table_paper['caller/truth_ratio']
table_paper['good/bluff_ratio'] = 1 - table_paper['good/truth_ratio']
table_paper['callee/bluff_ratio'] = 1 - table_paper['callee/truth_ratio']
table_paper.drop(['caller/truth_ratio', 'good/truth_ratio', 'callee/truth_ratio'], axis=1, inplace=True)
table_paper.sort_values(by='elo', ascending=False, inplace=True)
table_paper['model_bad_name'] = table_paper['model_bad_name'].str.replace('COMM', '')
table_paper


/tmp/ipykernel_7305/1143634429.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_7305/1143634429.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_7305/1143634429.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_730

,model_bad_name,elo,caller/bluff_ratio,good/bluff_ratio,callee/bluff_ratio
11,RNN h=32 m=64 P2,1358.348911,0.596476,0.618470,0.669566
16,heuristic,1354.971987,0.000000,0.000000,0.000000
13,RNN h=64 m=128 P2,1317.238421,0.000000,0.000000,0.000000
1,NN h=32 sample_batch=True P2,1292.638855,0.117039,0.141220,0.097077
15,RNN h=64 m=64 P2,1265.061225,0.003564,0.009396,0.005340
7,RNN h=128 m=64 P2,1259.962503,0.000000,0.000000,0.000000
9,RNN h=32 m=128 P2,1254.069632,0.004081,0.007628,0.006722
0,NN h=32 sample_batch=False P2,1245.837715,0.002804,0.004532,0.000641
5,RNN h=128 m=128 P2,1238.441849,0.000000,0.000000,0.000000
2,NN h=64 sample_batch=False P2,1171.546162,0.000009,0.000018,0.000009


In [ ]:
table_P2 = table_paper[table_paper['model_bad_name'].str.contains('P2')]
table_P2['callee/bluff_ratio'] = table_P2['callee/bluff_ratio'].round(2)
table_P2['caller/bluff_ratio'] = table_P2['caller/bluff_ratio'].round(2)
table_P2['good/bluff_ratio'] = table_P2['good/bluff_ratio'].round(2)
table_P2['elo'] = table_P2['elo'].round(0).astype(int)
table_P2['model_bad_name'] = table_P2['model_bad_name'].str.replace('P2', '')
print(table_P2.to_latex(index=False, float_format="%.2f", escape=False))

\begin{tabular}{lrrrr}
\toprule
model_bad_name & elo & caller/bluff_ratio & good/bluff_ratio & callee/bluff_ratio \\
\midrule
 RNN h=32 m=64  & 1358 & 0.60 & 0.62 & 0.67 \\
 RNN h=64 m=128  & 1317 & 0.00 & 0.00 & 0.00 \\
 NN h=32 sample_batch=True  & 1293 & 0.12 & 0.14 & 0.10 \\
 RNN h=64 m=64  & 1265 & 0.00 & 0.01 & 0.01 \\
 RNN h=128 m=64  & 1260 & 0.00 & 0.00 & 0.00 \\
 RNN h=32 m=128  & 1254 & 0.00 & 0.01 & 0.01 \\
 NN h=32 sample_batch=False  & 1246 & 0.00 & 0.00 & 0.00 \\
 RNN h=128 m=128  & 1238 & 0.00 & 0.00 & 0.00 \\
 NN h=64 sample_batch=False  & 1172 & 0.00 & 0.00 & 0.00 \\
 NN h=64 sample_batch=True  & 1143 & 0.01 & 0.02 & 0.01 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_7305/2467943868.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_7305/2467943868.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_7305/2467943868.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_7305